In [1]:
#library import
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
import os, random, json
spark=SparkSession.builder.master("local").appName("studentmarkpractice").getOrCreate()

In [2]:
class unification_to_refinement:
    
    """refinement"""
    
    def __init__(self,file_parameter):
        
        """function to get input parameters to read the file from local path"""
        self.file_parameter=file_parameter

    def write_parquet(self,df):
        df.write.format(self.file_parameter['dest_format']).mode("overwrite").option("path",self.file_parameter['dest_path']).saveAsTable("school_refinement.{0}".format(self.file_parameter['view_name']))
  
    def main_block(self):
        
        """main execution block"""
        temp_obj=common_ingestion(self.file_parameter)
        df=temp_obj.read_input_file()
        temp_obj.create_view(df)
    

In [3]:
if __name__=='__main__':
    file_parameter1={
        "src_format":"parquet",
        "src_path":"C:/Git_Repo/big_data_project/data_files/unified/school_unified/overallreport.parquet",
        "view_name":"school"
    }
    
    file_parameter2={
        "dest_format":"parquet",
        "dest_path":"C:/Git_Repo/big_data_project/data_files/refinement/school_refinement/highestmark.parquet",
        "view_name":"highestmark"
    }
    
    %run C:/Git_Repo/big_data_project/etl_scripts/common_ingestion/common_ingestion.ipynb
    
    spark.sql("create database if not exists {0}".format("school_refinement"))
    unified_obj1=unification_to_refinement(file_parameter1)
    unified_obj2=unification_to_refinement(file_parameter2)
    unified_obj1.main_block()
    sql_query=open("C:/Git_Repo/big_data_project/config/refinement_sql.sql","r")
    highmark_df=spark.sql(sql_query.read())
    unified_obj2.write_parquet(highmark_df)
    